In [35]:
import pandas as pd
#df = pd.read_json('watcher_crawl/watcher.jl', lines=True)
df = pd.read_json('../../watcher_crawl/watcher.jl', lines=True)

In [36]:
df.tail(5)

,company_name,company_code,company_sector,company_industry,scraped_table,records
20,reliance-industries-ltd,13215,Diversified,Diversified,income-quaterly,"{'columns': ['fields', 'Dec 2021', 'Sep 2021',..."
21,reliance-industries-ltd,13215,Diversified,Diversified,income-annual,"{'columns': ['fields', 'FY 2021', 'FY 2020',..."
22,reliance-industries-ltd,13215,Diversified,Diversified,balance-sheet,"{'columns': ['fields', 'FY 2021', 'FY 2020',..."
23,reliance-industries-ltd,13215,Diversified,Diversified,cash-flow,"{'columns': ['fields', 'FY 2021', 'FY 2020',..."
24,reliance-industries-ltd,13215,Diversified,Diversified,ratios,"{'columns': ['fields', 'FY 2021', 'FY 2020',..."


In [37]:

df['company_code'] = df['company_code'].astype(str)

In [38]:
df.head()

,company_name,company_code,company_sector,company_industry,scraped_table,records
0,adani-enterprises-ltd,9074,Diversified,Diversified,income-quaterly,"{'columns': ['fields', 'Sep 2021', 'Jun 2021',..."
1,adani-enterprises-ltd,9074,Diversified,Diversified,income-annual,"{'columns': ['fields', 'FY 2021', 'FY 2020',..."
2,adani-enterprises-ltd,9074,Diversified,Diversified,balance-sheet,"{'columns': ['fields', 'FY 2021', 'FY 2020',..."
3,adani-enterprises-ltd,9074,Diversified,Diversified,cash-flow,"{'columns': ['fields', 'FY 2021', 'FY 2020',..."
4,adani-enterprises-ltd,9074,Diversified,Diversified,ratios,"{'columns': ['fields', 'FY 2021', 'FY 2020',..."


In [39]:
table_names = list(df['scraped_table'])
print(table_names)
company_codes = list(df['company_code'])
print(company_codes)

['income-quaterly', 'income-annual', 'balance-sheet', 'cash-flow', 'ratios', 'income-quaterly', 'income-annual', 'balance-sheet', 'cash-flow', 'ratios', 'income-quaterly', 'income-annual', 'balance-sheet', 'cash-flow', 'ratios', 'income-quaterly', 'income-annual', 'balance-sheet', 'cash-flow', 'ratios', 'income-quaterly', 'income-annual', 'balance-sheet', 'cash-flow', 'ratios']
['9074', '9074', '9074', '9074', '9074', '2718', '2718', '2718', '2718', '2718', '11260', '11260', '11260', '11260', '11260', '9195', '9195', '9195', '9195', '9195', '13215', '13215', '13215', '13215', '13215']


In [40]:
import pymongo

In [41]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
m_db = myclient["The-watcher"]

In [44]:
df_collection = [] 
for i in range(df.shape[0]):
    company_code = company_codes[i]
    table_name = m_db[df['scraped_table'][i]]
    column_names = df['records'][i].get('columns')
    df2 = pd.DataFrame(df['records'][i].get('values'), columns = column_names)
    #print(df2.head())
    #df2.index = [str(i) for i in range(len(df2))]
    df2.set_index('fields',inplace=True)
    df3 = df2.T
    print(df3.head())
    
    df3.reset_index(inplace=True)
    df3['company_code'] = company_code
    table_name.insert_many(df3.to_dict('records'))
    for i in table_name.find():
        print(i)


fields   Total Income Total Income Growth (%) Total Expenses  \
Sep 2021    13,597.10                    6.81      12,655.00   
Jun 2021    12,730.63                   -7.00      11,907.96   
Mar 2021    13,688.95                   16.13      12,957.03   
Dec 2020    11,787.82                   26.59      11,060.12   
Sep 2020     9,312.14                   69.25       8,484.57   

fields   Total Expenses Growth (%)    EBIT EBIT Growth (%)  \
Sep 2021                      6.27  942.10           14.52   
Jun 2021                     -8.10  822.67           12.40   
Mar 2021                     17.15  731.92            0.58   
Dec 2020                     30.36  727.70          -12.07   
Sep 2020                     59.29  827.57          371.20   

fields   Profit after Tax (PAT) PAT Growth (%) EBIT Margin (%)  \
Sep 2021                 212.41         -21.75            6.93   
Jun 2021                 271.46          16.03            6.46   
Mar 2021                 233.95         -21.

In [53]:
collections = m_db.list_collection_names()
print(len(collections))

5


In [54]:
m_db.list_collection_names()


['income-annual', 'ratios', 'income-quaterly', 'cash-flow', 'balance-sheet']

In [61]:
coll = m_db['ratios']
coll.count_documents({})

25

In [62]:
sample_df = pd.DataFrame(list(coll.find()))
#print(sample_df.head())
print(sample_df[pd.isna(sample_df['Price to Sales (x)'])])

                         _id     index Return on Equity (%)  \
0   625502c6320499fde8bc0fb3  FY  2021                 5.37   
1   625502c6320499fde8bc0fb4  FY  2020                 6.71   
2   625502c6320499fde8bc0fb5  FY  2019                 4.86   
3   625502c6320499fde8bc0fb6  FY  2018                 5.01   
4   625502c6320499fde8bc0fb7  FY  2017                 6.98   
5   625502c6320499fde8bc0fcc  FY  2021               -25.58   
6   625502c6320499fde8bc0fcd  FY  2020               -41.71   
7   625502c6320499fde8bc0fce  FY  2019                 0.57   
8   625502c6320499fde8bc0fcf  FY  2018                 1.58   
9   625502c6320499fde8bc0fd0  FY  2017                 5.63   
10  625502c6320499fde8bc0fe5  FY  2021                11.97   
11  625502c6320499fde8bc0fe6  FY  2020                16.28   
12  625502c6320499fde8bc0fe7  FY  2019                20.28   
13  625502c6320499fde8bc0fe8  FY  2018                16.16   
14  625502c6320499fde8bc0fe9  FY  2017                1

In [63]:
sample_df.describe()

,_id,index,Return on Equity (%),Return on Capital Employed (%),Return on Assets (%),Interest Coverage Ratio (x),Asset Turnover Ratio (x),Price to Earnings (x),Price to Book (x),EV/EBITDA (x),EBITDA Margin (%),company_code,Cost to Income (%),Interest income to Earning assets (%),Interest Expense to Earning assets (%),Price to Sales (x),Operating Profit Margin (%)
count,25,25,25,20,25,20,20,25,25,20,20,25,5,5,5,5,5
unique,25,5,25,19,24,20,20,25,24,20,20,5,5,5,5,5,5
top,625502c6320499fde8bc0fb3,FY 2021,5.37,9.22,1.72,1.98,76.55,123.46,4.11,39.46,8.24,9074,41.55,7.14,3.29,6.40,3.51
freq,1,5,1,2,2,1,1,1,2,1,1,5,1,1,1,1,1


In [64]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   _id                                     25 non-null     object
 1   index                                   25 non-null     object
 2   Return on Equity (%)                    25 non-null     object
 3   Return on Capital Employed (%)          20 non-null     object
 4   Return on Assets (%)                    25 non-null     object
 5   Interest Coverage Ratio (x)             20 non-null     object
 6   Asset Turnover Ratio (x)                20 non-null     object
 7   Price to Earnings (x)                   25 non-null     object
 8   Price to Book (x)                       25 non-null     object
 9   EV/EBITDA (x)                           20 non-null     object
 10  EBITDA Margin (%)                       20 non-null     object
 11  company_

In [16]:
sample_df

fields,index,Total Income,Total Income Growth (%),Total Expenses,Total Expenses Growth (%),EBIT,EBIT Growth (%),Profit after Tax (PAT),PAT Growth (%),EBIT Margin (%),Net Profit Margin (%),Basic EPS (₹),company_code
0,Sep 2021,"13,597.10",6.81,"12,655.00",6.27,942.10,14.52,212.41,-21.75,6.93,1.56,1.93,9074
1,Jun 2021,"12,730.63",-7.00,"11,907.96",-8.10,822.67,12.40,271.46,16.03,6.46,2.13,2.47,9074
2,Mar 2021,"13,688.95",16.13,"12,957.03",17.15,731.92,0.58,233.95,-21.18,5.35,1.71,2.13,9074
3,Dec 2020,"11,787.82",26.59,"11,060.12",30.36,727.70,-12.07,296.81,-18.09,6.17,2.52,2.70,9074
4,Sep 2020,"9,312.14",69.25,"8,484.57",59.29,827.57,371.20,362.37,"1,116.41",8.89,3.89,3.30,9074


In [67]:
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__
list(imports())

['builtins', 'builtins', 'pandas', 'pymongo', 'types']